In [ ]:
import airfoil_simulator
import tensorflow as tf
import numpy as np
import lstm_model
import time
from skopt import gp_minimize
import visualize as viz

In [ ]:
def obj_airfoil_lift_drag(x):
    x = np.array(x)
    obj_value = -1*airfoil_simulator.objective(x.reshape(-1))/100
    return obj_value

In [ ]:
# imp.reload(lstm_model) # remove this when it's ready
steps = 20
dim = 2
x_0 = np.array([-0.1,-0.1]).reshape(-1,2)

In [ ]:
def optimize_lstm(x_0, steps):
    #load best model
    best_model_2d='rnn-cell-2d-1498485717'   
    print("Starting at %s and optimize for %d steps" % (x_0, steps) )
    sess, model_params = lstm_model.load_trained_model(best_model_2d)
    
    start_time = time.time()
    samples_x, samples_y = lstm_model.generate_sample_sequence(sess, model_params, x_0, steps = steps, \
        obj_func=lambda x: np.array(obj_airfoil_lift_drag(x)).reshape(1,-1) \
    )

    sample_y = np.array(sample_y).flatten()

    print("Min -L/D found %f at %s" % (sample_y[min_y_idx], sample_x[min_y_idx].flatten()) )
    print("Took %f mins" % (( time.time() - start_time ) / 60.0) )
    
    return samples_x, samples_y

def optimize_random(x_0, steps):
    samples_x = [x_0]
    samples_y = [obj_airfoil_lift_drag(x_0)]
    for i in range(steps):
        #print(i)
        x = np.random.uniform(low=-1.0, high=1.0, size=2)
        y = obj_airfoil_lift_drag(x)
        
        samples_x += [x]
        samples_y += [y]
        
    return samples_x, samples_y

def optimize_skopt(x_0, steps):
    res = gp_minimize(obj_airfoil_lift_drag, dimensions=[(-1.0,1.0)]*2, n_calls=steps, n_random_starts=10, x0=x_0) 
    return res.xiters, res.funcvals

In [ ]:
#samples_x_lstm, samples_y_lstm = optimize_lstm()
samples_x_random, samples_y_random = optimize_random(x_0, steps)
#samples_x_skopt, samples_y_skopt = optimize_skopt()

In [ ]:
viz.plot_min_curves([sample_y[np.newaxis,:], np.array(y_random)[np.newaxis,:], res.func_vals[np.newaxis,:]],['MIN', 'RANDOM', 'SKOPT'])